Assignment 5
Validation and Regularization
Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)

In [ ]:
htrain=pd.read_csv('healthcareTrain.csv')
htest=pd.read_csv('healthcareTest.csv')

In [ ]:
htrain.head(5)


In [ ]:
htest.head(5)


Part 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')
accDf=pd.DataFrame()

df=htrain[["total_los","num_op","num_er","num_ndc","pre_total_cost", "pre_CCI","pdc_80_flag"]]
#Scale the input features
sc = MinMaxScaler()
df.iloc[:, :-1]= sc.fit_transform(df.iloc[:, :-1])
#trainn = (trainn - trainn.min()) / (trainn.max() - trainn.min())

k = 10
kFoldsDf = np.array_split(df, k)

for i in range(k):
    # Save a copy, to be modified for each iteration.
    trainDf = kFoldsDf.copy() 
    # Set aside the ith fold for validation.
    valDf = trainDf[i]
    # Delete the validation fold from training, for this training session
    del(trainDf[i])
    # Concatenate the remaining folds into one, for this training session 
    trainDf = pd.concat(trainDf, sort=False)
    train_X = trainDf.iloc[:,:-1]
    train_y = trainDf.iloc[:,-1]
    val_X = valDf.iloc[:,:-1]
    val_y = valDf.iloc[:,-1]

    kn_range=np.arange(31,103,2)
    for kn in kn_range:
        model = KNeighborsClassifier(n_neighbors=kn,p=2)
        model.fit(train_X, train_y)
        # accDf gets filled row by row. First row is the accuracy of the first fold from 31NN to 103NN.
        #Every row in accDf corresponds to a fold. Every column in accDf corresponds to a the number of neighbors (kn).
        accDf.loc[i,str(kn)+'NN']=accuracy_score(val_y,model.predict(val_X))

accDf

In [ ]:
cv_accDf = pd.DataFrame(np.round(accDf.mean(axis='rows'),4))
cv_accDf.columns = ['CV Accuracy']
print('Cross Validation Accuracy for different number of neighbors are:\n', cv_accDf)

Part 2

In [ ]:
fig = px.line(cv_accDf, x=np.arange(31,103,2), y='CV Accuracy', markers = 'True')
fig.update_layout(xaxis_title = 'Number of neighbors (K)')
fig.show()

Part 3

In [ ]:
train_XX=htrain[["total_los","num_op","num_er","num_ndc","pre_total_cost", "pre_CCI"]]
train_yy=htrain['pdc_80_flag']

test_XX=htest[["total_los","num_op","num_er","num_ndc","pre_total_cost", "pre_CCI"]]
test_yy=htest['pdc_80_flag']

norm=MinMaxScaler()
train_XX = norm.fit_transform(train_XX)
test_XX=norm.transform(test_XX)

model2 = KNeighborsClassifier(n_neighbors=31,p=2)
model2.fit(train_XX, train_yy)
pred_yy = model2.predict(test_XX)
acc2=accuracy_score(test_yy,pred_yy)
print("Accuracy:",acc2,"(",round((acc2)*100,2),"%",").")

In [ ]:
validation_error=1-best_acc
test_eror=1-acc2

Comparison
Accuracy - Error DataFrame

In [ ]:
acc_err_df=pd.DataFrame({"Set":["Validation","Test"],
                        "Accuracy":[round(cv_accDf['CV Accuracy'].max(),3),round(acc2,3)],
                        })
acc_err_df
